In [ ]:
pip install langchain wikipedia-api openai python-dotenv

In [ ]:
pip install wikipedia

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

# ✅ Initialize Wikipedia tool (No API key required)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

def wikipedia_chatbot():
    print("Welcome to the Wikipedia Chatbot! Type 'exit' to quit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye!")
            break

        try:
            response = wikipedia.run(user_input)  # 🔍 Fetch answer from Wikipedia
            print(f"Chatbot: {response}")
        except Exception as e:
            print(f"Chatbot: Error encountered: {e}")

if __name__ == "__main__":
    wikipedia_chatbot()


Welcome to the Wikipedia Chatbot! Type 'exit' to quit.
You: What is a black hole
Chatbot: Page: Black hole
Summary: A black hole is a massive, compact astronomical object so dense that its gravity prevents anything from escaping, even light. Albert Einstein's theory of general relativity predicts that a sufficiently compact mass will form a black hole. The boundary of no escape is called the event horizon. A black hole has a great effect on the fate and circumstances of an object crossing it, but has no locally detectable features according to general relativity. In many ways, a black hole acts like an ideal black body, as it reflects no light.  Quantum field theory in curved spacetime predicts that event horizons emit Hawking radiation, with the same spectrum as a black body of a temperature inversely proportional to its mass. This temperature is of the order of billionths of a kelvin for stellar black holes, making it essentially impossible to observe directly.
Objects whose gravitat

In [ ]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 47.8 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
import os

def extract_text_from_pdf(pdf_path):
    """Extract text from a given PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

def create_faiss_index(text, model_name='all-MiniLM-L6-v2'):
    """Create a FAISS index from extracted text."""
    model = SentenceTransformer(model_name)
    sentences = text.split(". ")  # Splitting into meaningful chunks
    embeddings = model.encode(sentences)

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings, dtype=np.float32))

    return index, sentences, model

def retrieve_answer(query, index, sentences, model, top_k=1):
    """Retrieve the most relevant short answer from FAISS index."""
    query_embedding = model.encode([query])
    _, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
    return sentences[indices[0][0]] if indices[0][0] < len(sentences) else "No relevant answer found."

if __name__ == "__main__":
    pdf_path = "/content/ch1.pdf"  # Change this to your PDF file
    text = extract_text_from_pdf(pdf_path)
    index, sentences, model = create_faiss_index(text)

    while True:
        query = input("Ask a question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        answer = retrieve_answer(query, index, sentences, model)
        print(answer)


Ask a question (or type 'exit' to quit): What do you mean by Bootstrap program?
  

Everything else is either 
a system program (ships with the operating system) , or 
an application program
Ask a question (or type 'exit' to quit): what are the dual mode?
virtual machine manager (VMM) mode for guest VMs 
 

1.33 
Silberschatz, Galvin and Gagne ©2013 
Operating System Concepts – 9th Edition 
Transition from User to Kernel Mode 

Timer to prevent infinite loop / process hogging resources 
Timer is set to interrupt the computer after some time period 
Keep a counter that is decremented by the physical clock
Ask a question (or type 'exit' to quit): physical clock means
virtual machine manager (VMM) mode for guest VMs 
 

1.33 
Silberschatz, Galvin and Gagne ©2013 
Operating System Concepts – 9th Edition 
Transition from User to Kernel Mode 

Timer to prevent infinite loop / process hogging resources 
Timer is set to interrupt the computer after some time period 
Keep a counter tha

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel, WhisperProcessor, WhisperForConditionalGeneration
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from PIL import Image
import torchaudio
import os

# Load models
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-base")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
text_encoder = SentenceTransformer("all-mpnet-base-v2")  # Improved model for better retrieval

# FAISS index for retrieval
d = 768  # Updated dimensionality for new embedding model
index = faiss.IndexFlatL2(d)
metadata_store = []  # Store metadata alongside indexed data

# Function to extract features from text
def encode_text(text):
    return text_encoder.encode(text, normalize_embeddings=True, convert_to_numpy=True)

# Function to extract features from image
def encode_image(image_path):
    if not os.path.exists(image_path):
        return None
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features.squeeze().numpy()

# Function to extract features from audio
def encode_audio(audio_path):
    if not os.path.exists(audio_path):
        return None
    audio, sr = torchaudio.load(audio_path)
    inputs = whisper_processor(audio, return_tensors="pt", sampling_rate=sr)
    with torch.no_grad():
        generated_ids = whisper_model.generate(inputs.input_features)
        transcript = whisper_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return encode_text(transcript)

# Add data to FAISS index
def add_to_index(data, metadata):
    vectors = []
    valid_metadata = []
    for item, meta in zip(data, metadata):
        if isinstance(item, str):
            vector = encode_text(item)
        elif isinstance(item, np.ndarray):
            vector = item
        else:
            continue
        vectors.append(vector)
        valid_metadata.append(meta)

    if vectors:
        vectors = np.array(vectors)
        index.add(vectors)
        metadata_store.extend(valid_metadata)

# Retrieve relevant data based on query
def retrieve(query, k=5):
    query_vector = encode_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, k)
    return [metadata_store[i] for i in indices[0] if i < len(metadata_store)]

# General multimodal retrieval function
def multimodal_rag(query):
    return retrieve(query)

# Function to process user input and add relevant data
def process_and_add_data():
    while True:
        data_type = input("Enter data type (text/image/audio) or 'done' to finish: ").strip().lower()
        if data_type == 'done':
            break
        elif data_type == 'text':
            text = input("Enter text data: ")
            add_to_index([text], [text])
        elif data_type == 'image':
            image_path = input("Enter image file path: ")
            image_vector = encode_image(image_path)
            if image_vector is not None:
                add_to_index([image_vector], [image_path])
        elif data_type == 'audio':
            audio_path = input("Enter audio file path: ")
            audio_vector = encode_audio(audio_path)
            if audio_vector is not None:
                add_to_index([audio_vector], [audio_path])
        else:
            print("Invalid data type. Please enter 'text', 'image', or 'audio'.")

# Allow user to add custom data before querying
print("Add your custom data before querying.")
process_and_add_data()

# Continuous query loop
while True:
    query = input("Enter your query (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    response = multimodal_rag(query)
    print("Response:", response)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Add your custom data before querying.
Enter data type (text/image/audio) or 'done' to finish: image
Enter image file path: "/content/dog.avif"
Enter data type (text/image/audio) or 'done' to finish: text
Enter text data: What is a black hole
Enter data type (text/image/audio) or 'done' to finish: done
Enter your query (or type 'exit' to quit): exit
